In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

file_path = 'dataset.xlsx'

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(file_path)

In [18]:
# Extract unique flags from the "FLC: REDEN VERWERPING" column
all_flags = df["FLC: REDEN VERWERPING"].str.split('|', expand=True).stack().str.strip().unique()

# Print the unique flags
print("Unique Flags:")
for flag in all_flags:
    print(flag)

Unique Flags:
The cost was declared in the wrong category
double declaration
please specify the correct invoiced amount
Please specify the correct invoiced amount
Costs that are declared too late, can only be reimbursed for 50%
Purchase not eligible for subsidy
please specify the correct supplier


In [19]:
# Define the phrases to check for in the specified column
phrases = {
    "FLC: REDEN VERWERPING": {
        "The cost was declared in the wrong category": "category",
        "double declaration": "double",
        "correct invoiced amount": "amount",
        "too late": "late",
        "eligible": "eligible",
        "supplier": "supplier"
    }
}

# Iterate through the phrases and update the corresponding columns
for column_name, conditions in phrases.items():
    for phrase, new_column in conditions.items():
        df[new_column] = df[column_name].str.contains(phrase, case=False, na=False).astype(int)

In [21]:
# Convert the "DATUM FACTUUR - DECLARED" column to datetime format
df['DATUM FACTUUR - DECLARED'] = pd.to_datetime(df['DATUM FACTUUR - DECLARED'], errors='coerce')

# Create the "latest" column by moving the month 6 months forward
df['latest'] = df['DATUM FACTUUR - DECLARED'] + pd.DateOffset(months=6)

    DATUM FACTUUR - DECLARED     latest
0                 2022-12-23 2023-06-23
1                 2020-12-12 2021-06-12
2                 2018-04-25 2018-10-25
3                 2021-12-28 2022-06-28
4                 2021-05-08 2021-11-08
..                       ...        ...
489               2022-04-05 2022-10-05
490               2019-02-16 2019-08-16
491               2020-06-12 2020-12-12
492               2021-06-20 2021-12-20
493               2020-04-13 2020-10-13

[494 rows x 2 columns]


In [22]:
# Convert date columns to datetime objects
df['latest'] = pd.to_datetime(df['latest'])
df['DECLARATIEDATUM (can be assumed to be close to payment date)'] = pd.to_datetime(df['DECLARATIEDATUM (can be assumed to be close to payment date)'])

# Calculate the time difference and create a new column 'timediff'
df['timediff'] = df['DECLARATIEDATUM (can be assumed to be close to payment date)'] - df['latest']

        latest DECLARATIEDATUM (can be assumed to be close to payment date)  \
0   2023-06-23                                         2023-06-18             
1   2021-06-12                                         2021-06-07             
2   2018-10-25                                         2018-09-30             
3   2022-06-28                                         2022-02-20             
4   2021-11-08                                         2021-10-18             
..         ...                                                ...             
489 2022-10-05                                         2022-06-25             
490 2019-08-16                                         2019-02-25             
491 2020-12-12                                         2020-10-26             
492 2021-12-20                                         2021-10-01             
493 2020-10-13                                         2020-07-26             

     timediff  
0     -5 days  
1     -5 days  
2  

In [24]:
# Calculate the time difference and create a new column 'timediff'
df['amountdiff'] = df['BETAALD BEDRAG - extracted from invoice'] - df['BETAALD BEDRAG - declared ']

In [32]:
# Create the "suppdiff" column by comparing "LEVERANCIER - EXTRACTED" and "LEVERANCIER - DECLARED"
df['suppdiff'] = df['LEVERANCIER - EXTRACTED'] != df['LEVERANCIER - DECLARED']

# Print the DataFrame with the new "suppdiff" column
print(df[['LEVERANCIER - EXTRACTED', 'LEVERANCIER - DECLARED', 'suppdiff']])


         LEVERANCIER - EXTRACTED       LEVERANCIER - DECLARED  suppdiff
0                   Quick Travel                 Quick Travel     False
1                       Taxi Joe                     Taxi Joe     False
2    Barker and Barker analytics  Barker and Barker analytics     False
3                       Law Firm                     Law Firm     False
4          Scientific Equipment         Scientific Equipment      False
..                           ...                          ...       ...
489         BrochurePrinters Co.         BrochurePrinters Co.     False
490            AeroLink Airlines            AeroLink Airlines     False
491      EconoMetrics Consulting      EconoMetrics Consulting     False
492             ArcheoTools Ltd.             ArcheoTools Ltd.     False
493   CommunityBuild Renovations   CommunityBuild Renovations     False

[494 rows x 3 columns]


In [37]:
df2 = pd.read_excel('output_file.xlsx')

# Filter rows where "category" column has value 0
df2 = df2[df2['category'] == 0]

# Save the filtered DataFrame to a CSV file
df2.to_csv('cat_data.csv', index=False)

In [38]:
# Assuming your columns are named 'BESCHRIJVING DECLARATIE' and 'KOSTENRUBRIEK - declared'
X = df2['BESCHRIJVING DECLARATIE']
y = df2['KOSTENRUBRIEK - declared']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a Multinomial Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test set
predictions = classifier.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
classification_rep = classification_report(y_test, predictions)

print(f'Accuracy: {accuracy:.2f}')
print('\nClassification Report:')
print(classification_rep)

Accuracy: 0.89

Classification Report:
                             precision    recall  f1-score   support

                  Equipment       0.95      0.95      0.95        19
          External Services       0.83      0.91      0.87        22
Infrastructure and building       0.94      0.94      0.94        16
          Preparation Costs       0.00      0.00      0.00         1
          Preparation costs       0.84      0.76      0.80        21
               Travel costs       0.94      1.00      0.97        16

                   accuracy                           0.89        95
                  macro avg       0.75      0.76      0.75        95
               weighted avg       0.89      0.89      0.89        95



c:\Users\camd1\miniconda3\envs\holyhack\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\camd1\miniconda3\envs\holyhack\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\camd1\miniconda3\envs\holyhack\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [28]:
# Save the updated DataFrame to a new Excel file
output_file_path = 'output_file.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Updated Excel file saved to: {output_file_path}")

Updated Excel file saved to: output_file.xlsx
